In [23]:
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam

# Directorios de datos
train_dir = 'C:\\Eder\\Cosas TEC\\9 Semestre\\Inteligencia Artificial\\Dataset flores\\Train'
test_dir = 'C:\\Eder\\Cosas TEC\\9 Semestre\\Inteligencia Artificial\\Dataset flores\\Test'

# Parámetros del modelo
img_width, img_height = 150, 150
input_shape = (img_width, img_height, 3)
epochs = 10
batch_size = 16  # Intenta con 16 o 8, ajusta según sea necesario

# Generadores de datos con aumento para entrenamiento
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255.0,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(rescale=1.0 / 255.0)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical'
)

# Imprime el número de muestras de entrenamiento antes de model.fit()
print("Número de muestras de entrenamiento:", train_generator.samples)

# Asegurarse de que hay al menos una muestra de entrenamiento y que batch_size es un divisor de train_generator.samples
assert train_generator.samples > 0, "Número de muestras de entrenamiento es cero"
assert train_generator.samples % batch_size == 0, "batch_size no es un divisor exacto de train_generator.samples"

# Construir el modelo CNN
model = Sequential()

model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(len(train_generator.class_indices), activation='softmax'))

model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Entrenar el modelo
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs,
    validation_data=test_generator,
    validation_steps=test_generator.samples // batch_size
)

# Guardar el modelo
model.save('modelo_flores.keras')





Found 400 images belonging to 5 classes.
Found 7 images belonging to 5 classes.
Número de muestras de entrenamiento: 400


C:\Users\esque\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Epoch 1/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 7s 208ms/step - accuracy: 0.3105 - loss: 1.5047 - val_accuracy: 0.7143 - val_loss: 1.0100
Epoch 2/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.7143 - val_loss: 1.0100
Epoch 3/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 6s 211ms/step - accuracy: 0.5571 - loss: 1.0346 - val_accuracy: 1.0000 - val_loss: 0.6179
Epoch 4/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.6179
Epoch 5/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 7s 245ms/step - accuracy: 0.7116 - loss: 0.7326 - val_accuracy: 0.7143 - val_loss: 0.6517
Epoch 6/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.7143 - val_loss: 0.6517
Epoch 7/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 7s 246ms/step - accuracy: 0.7489 - loss: 0.6199 - val_accuracy: 0.7143 - val_loss: 0.4614
Epoch 8/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.0000e+00 - loss: 0.000

In [35]:
import os
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np

# Directorio donde se encuentra el modelo (.h5)
modelo_ruta = 'C:\Eder\Cosas TEC\9 Semestre\Inteligencia Artificial\modelo_flores.keras'

# Cargar el modelo
modelo = load_model(modelo_ruta)

# Directorio donde se encuentran las nuevas imágenes de prueba
directorio_base = 'C:\Eder\Cosas TEC\9 Semestre\Inteligencia Artificial\Dataset flores\Test'

# Diccionario de mapeo de clases a índices (deberías tener este mapeo basado en el entrenamiento)
mapeo_clases_a_indices = {'Bluebell': 0, 'Daisy': 1, 'Fritillary': 2, 'Sunflower': 3, 'Tigerlily': 4}

# Invertir el diccionario para obtener un mapeo de índices a nombres de clases
mapeo_indices_a_clases = {v: k for k, v in mapeo_clases_a_indices.items()}

# Recorrer todas las subcarpetas y archivos en el directorio base
for subdir, _, files in os.walk(directorio_base):
    for image_file in files:
        # Cargar la imagen y preprocesarla
        img_path = os.path.join(subdir, image_file)
        img = image.load_img(img_path, target_size=(150, 150))
        img_array = image.img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)
        img_array /= 255.0  # Normalizar la imagen

        # Realizar la predicción
        predicciones = modelo.predict(img_array)

        # Obtener la etiqueta predicha
        etiqueta_predicha = np.argmax(predicciones, axis=1)

        # Obtener el nombre de la clase a partir del índice predicho
        nombre_clase_predicha = mapeo_indices_a_clases[etiqueta_predicha[0]]

        print(f'Archivo: {img_path}, Clase predicha: {nombre_clase_predicha}')



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
Archivo: C:\Eder\Cosas TEC\9 Semestre\Inteligencia Artificial\Dataset flores\Test\Bluebell\Bluebell (Hyacinthoides non-scripta)_0.jpg, Clase predicha: Bluebell
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
Archivo: C:\Eder\Cosas TEC\9 Semestre\Inteligencia Artificial\Dataset flores\Test\Bluebell\Close-Up-of-Dark-Blue-Flowers.jpg, Clase predicha: Bluebell
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
Archivo: C:\Eder\Cosas TEC\9 Semestre\Inteligencia Artificial\Dataset flores\Test\Daisy\image_0815.jpg, Clase predicha: Daisy
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
Archivo: C:\Eder\Cosas TEC\9 Semestre\Inteligencia Artificial\Dataset flores\Test\Daisy\image_0855.jpg, Clase predicha: Daisy
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
Archivo: C:\Eder\Cosas TEC\9 Semestre\Inteligencia Artificial\Dataset flores\Test\Fritillary\44d7c319d06b6efbf59cd8f184505130.jpg, Clase predicha: Fritillary
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
Archivo: C:\Eder\Cosas TEC\9 Semestre\Inteligencia 